<a href="https://colab.research.google.com/github/allenhu08/capstone/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/allenhu08/capstone.git || cd capstone && git pull

Cloning into 'capstone'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 6), reused 15 (delta 2), pack-reused 0
Unpacking objects: 100% (27/27), done.
fatal: not a git repository (or any of the parent directories): .git


In [2]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('./capstone/datasets/rawdata_new.csv')


In [15]:
print(df.columns.tolist())

['id', 'date', 'total_agr_land', 'workforce', 'trees', 'arable_land', 'ley', 'perm_grass', 'perm_crops', 'prot_cult', 'add_agr_land', 'oth_land', 'qual_landscape', 'spec_crops', 'grapes', 'vegetables', 'fruits', 'oth_spec_crops', 'animal_unit', 'cattle_animal_unit', 'dairy_cows', 'suckler_cows', 'hor_shee_goa', 'pigs_poultry', 'pigs', 'poultry', 'eco_proof', 'organic', 'farmtype', 'region', 'age', 'educ', 'cons_general', 'percep_hail', 'percep_drought', 'percep_frost', 'percep_heavyrain', 'percep_longrain', 'percep_heat', 'cons_farm', 'self_reduc', 'self_act', 'self_cap', 'self_conf', 'self_not', 'legum', 'legum_eff', 'conc', 'conc_eff', 'add', 'add_eff', 'lact', 'lact_eff', 'breed', 'breed_eff', 'covman', 'covman_eff', 'comp', 'comp_eff', 'drag', 'drag_eff', 'cov', 'cov_eff', 'plough', 'plough_eff', 'solar', 'solar_eff', 'biog', 'biog_eff', 'ecodr', 'ecodr_eff', 'legum_imag', 'conc_imag', 'add_imag', 'lact_imag', 'breed_imag', 'covman_imag', 'comp_imag', 'drag_imag', 'cov_imag', 'plou

In [40]:
farm_info_cols = ['total_agr_land', 'workforce', 'trees', 'arable_land', 'ley', 'perm_grass', 'perm_crops', 'prot_cult', 'add_agr_land', 'oth_land', 'qual_landscape', 'eco_proof', 'organic',  'age'] #'farmtype', 'region',
crops_info_cols = ['spec_crops', 'grapes', 'vegetables', 'fruits', 'oth_spec_crops']
animal_info_cols = ['animal_unit', 'cattle_animal_unit', 'dairy_cows', 'suckler_cows', 'hor_shee_goa', 'pigs_poultry', 'pigs', 'poultry']
attitude_info_cols = ['educ', 'cons_general', 'percep_hail', 'percep_drought', 'percep_frost', 'percep_heavyrain', 'percep_longrain', 'percep_heat', 'cons_farm', 'self_reduc', 'self_act', 'self_cap', 'self_conf', 'self_not']
meansures_info_cols = ['legum', 'legum_eff', 'conc', 'conc_eff', 'add', 'add_eff', 'lact', 'lact_eff', 'breed', 'breed_eff', 'covman', 'covman_eff', 'comp', 'comp_eff', 'drag', 'drag_eff', 'cov', 'cov_eff', 'plough', 'plough_eff', 'solar', 'solar_eff', 'biog', 'biog_eff', 'ecodr', 'ecodr_eff']

df = df[farm_info_cols + crops_info_cols + animal_info_cols + meansures_info_cols]
df = df.fillna(df.mean())
df.head()



,total_agr_land,workforce,trees,arable_land,ley,perm_grass,perm_crops,prot_cult,add_agr_land,oth_land,qual_landscape,eco_proof,organic,age,spec_crops,grapes,vegetables,fruits,oth_spec_crops,animal_unit,cattle_animal_unit,dairy_cows,suckler_cows,hor_shee_goa,pigs_poultry,pigs,poultry,legum,legum_eff,conc,conc_eff,add,add_eff,lact,lact_eff,breed,breed_eff,covman,covman_eff,comp,comp_eff,drag,drag_eff,cov,cov_eff,plough,plough_eff,solar,solar_eff,biog,biog_eff,ecodr,ecodr_eff
0,4925.0,1.95406,79,3871,107,793,140,0,0,165,0,1.0,0.0,62.0,140,140,0,0,0,3.9250,3.9250,0.0000,0.9973,0.0000,0.00,0.0,0.00,2,3.294118,2,3.094118,2,3.418919,3,2.75,2,2.409836,2,3.546512,2,2.68,1,5.000000,1,4.000000,2,2.547368,2,3.516854,2,2.772152,1,3.0
1,2176.0,6.61310,12,1671,0,98,338,119,0,93,0,1.0,0.0,53.0,2088,0,1790,0,338,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.0,0.00,3,3.294118,3,3.094118,3,3.418919,3,2.75,3,2.409836,3,3.546512,3,2.68,2,3.666667,2,3.086957,2,1.000000,2,4.000000,2,2.772152,1,4.0
2,2251.0,1.39384,31,1507,279,380,81,0,4,512,0,1.0,0.0,60.0,81,81,0,0,0,15.9528,15.9028,14.6137,0.0000,0.0000,0.05,0.0,0.05,2,4.000000,2,1.000000,2,1.000000,1,1.00,2,2.000000,2,3.000000,2,1.00,2,3.000000,2,1.000000,2,1.000000,2,3.000000,2,2.000000,1,3.0
3,2389.0,1.99523,3,1969,71,322,0,0,27,350,0,1.0,0.0,62.0,0,0,0,0,0,52.9000,0.0000,0.0000,0.0000,0.0000,52.90,0.0,52.90,2,3.000000,2,3.000000,3,4.000000,3,2.00,3,2.000000,2,3.000000,2,2.00,2,2.000000,1,4.000000,2,2.000000,2,4.000000,2,4.000000,2,2.0
4,3205.0,3.09942,39,1911,675,461,117,0,14,112,0,1.0,0.0,57.0,117,0,0,0,117,56.5108,54.9184,40.3041,0.0000,1.3424,0.25,0.0,0.25,2,3.000000,2,3.000000,2,3.000000,2,1.00,2,1.000000,1,3.000000,2,1.00,2,1.000000,2,1.000000,2,1.000000,2,3.000000,2,2.000000,1,3.0


In [55]:

from sklearn.decomposition import TruncatedSVD
n_factors = len(meansures_info_cols) // 2
svd = TruncatedSVD(n_components=n_factors,random_state=42)
u_sigma = svd.fit_transform(df)

vt = svd.components_
sigma = svd.singular_values_

u = u_sigma.dot(np.linalg.inv(np.diag(sigma)))
p = u * np.sqrt(sigma)
q = np.transpose(vt) * np.sqrt(sigma)

In [51]:
test_sample = df.sample()

pred = np.array([np.dot(test_sample, q)])
print(pred)
print(test_sample[meansures_info_cols])

[[[ 7.57070063e+05 -2.00343807e+05  2.51140369e+04 -2.54284816e+04
    1.77823859e+04 -1.45195167e+03 -9.25691134e+03 -1.67280641e+03
   -1.07686962e+03  2.78446349e+02 -2.39363386e+02  8.16022143e+01
   -4.37500854e+02]]]
    legum  legum_eff  conc  conc_eff  add  add_eff  lact  lact_eff  breed  \
70      2        2.0     2       2.0    1      5.0     2       1.0      2   

    breed_eff  covman  covman_eff  comp  comp_eff  drag  drag_eff  cov  \
70        2.0       2         2.0     2       2.0     1       3.0    1   

    cov_eff  plough  plough_eff  solar  solar_eff  biog  biog_eff  ecodr  \
70      3.0       2         2.0      1        4.0     2       3.0      1   

    ecodr_eff  
70        5.0  
